# FORCING

# So Idea:
- get typical (aggregated) forcing for both regimes (100m box model structure)
- instead of aggregating to one year, get forcing for 5 years per each regime


# now what to do first: 
1. get necessary raw data, 
2. then integrate over entire time-series & depth (if necessary)
3. create aggregated forcing raw csvs for model (simply include proper Date information, in addition to yday)
4. date truncation & aggregation can happen in python code of model!

In [2]:
### create one large file with all possible forcing? (depends on where data is from, don't overcomplicate)

Let's write up what I need:
1. x25.8 depth
2. T_box
3. PAR
4. N0 (mean 100-150m)
4. NH40, ....
5. Si0
6. P0


In [3]:
### important point: actually use (as much as possible) raw values as forcing, then assumptions more apparent

# NISKIN DATA

In [22]:
require(ncdf4, warn.conflicts = FALSE);
require(cowplot, warn.conflicts = FALSE);
require(tidyverse, warn.conflicts = FALSE);
require(oce, warn.conflicts = FALSE);

Loading required package: oce
Loading required package: testthat

Attaching package: ‘testthat’

The following object is masked from ‘package:dplyr’:

    matches

The following object is masked from ‘package:purrr’:

    is_null

Loading required package: gsw


In [5]:
# open a NetCDF file
niskdat <- nc_open("../01Observations/Data/niskin.netcdf")
#print(niskdat)

In [42]:
niskdat_df <- data.frame("Num" = 1:niskdat$dim$unlimited$len)
print('start')
for(i in 1:niskdat$nvars){
    #print(i)
    niskdat_df[paste(attributes(niskdat$var)$names[i])] <- ncvar_get(niskdat, attributes(niskdat$var)$names[i])
}
print('done reading')
head(niskdat_df)

[1] "start"
[1] "done reading"


Num,Cruise_number,Cruise_ID_1,Cruise_ID_2,Leg,Day,Month,Year,Latitude,Longitude,...,Bact_Biomass_uMC,Bio_cast_no,ISO_DateTime_start_hc_local,ISO_DateTime_end_hc_local,ISO_DateTime_start_bc_local,ISO_DateTime_end_bc_local,ISO_DateTime_start_hc_UTC,ISO_DateTime_end_hc_UTC,ISO_DateTime_start_bc_UTC,ISO_DateTime_end_bc_UTC
1,1,93HG_001,CAR-001,2,8,11,1995,10.5,-64.667,...,BM_master,2,1995-11-08T06:12:00,1995-11-08T06:38:00,1995-11-08T03:36:00,1995-11-08T03:54:00,1995-11-08T10:12:00,1995-11-08T10:38:00,1995-11-08T07:36:00,1995-11-08T07:54:00
2,1,93HG_001,CAR-001,2,8,11,1995,10.5,-64.667,...,BM_master,2,1995-11-08T06:12:00,1995-11-08T06:38:00,1995-11-08T03:36:00,1995-11-08T03:54:00,1995-11-08T10:12:00,1995-11-08T10:38:00,1995-11-08T07:36:00,1995-11-08T07:54:00
3,1,93HG_001,CAR-001,2,8,11,1995,10.5,-64.667,...,BM_master,2,1995-11-08T06:12:00,1995-11-08T06:38:00,1995-11-08T03:36:00,1995-11-08T03:54:00,1995-11-08T10:12:00,1995-11-08T10:38:00,1995-11-08T07:36:00,1995-11-08T07:54:00
4,1,93HG_001,CAR-001,2,8,11,1995,10.5,-64.667,...,BM_master,2,1995-11-08T06:12:00,1995-11-08T06:38:00,1995-11-08T03:36:00,1995-11-08T03:54:00,1995-11-08T10:12:00,1995-11-08T10:38:00,1995-11-08T07:36:00,1995-11-08T07:54:00
5,1,93HG_001,CAR-001,2,8,11,1995,10.5,-64.667,...,BM_master,2,1995-11-08T06:12:00,1995-11-08T06:38:00,1995-11-08T03:36:00,1995-11-08T03:54:00,1995-11-08T10:12:00,1995-11-08T10:38:00,1995-11-08T07:36:00,1995-11-08T07:54:00
6,1,93HG_001,CAR-001,2,8,11,1995,10.5,-64.667,...,BM_master,2,1995-11-08T06:12:00,1995-11-08T06:38:00,1995-11-08T03:36:00,1995-11-08T03:54:00,1995-11-08T10:12:00,1995-11-08T10:38:00,1995-11-08T07:36:00,1995-11-08T07:54:00


In [43]:
niskdat_df$Date <- paste(niskdat_df$Year,'-',niskdat_df$Month,'-',niskdat_df$Day,sep='')

In [44]:
niskdat_df$date <- as.Date(niskdat_df$Date) # as.Date(substr(niskdat_df$ISO_DateTime_end_hc_UTC, 0, 10))

In [45]:
niskdat_df$Depth_real <- as.numeric(niskdat_df$Depth_real)
niskdat_df$Depth_target <- as.numeric(niskdat_df$Depth_target)

niskdat_df$Temperature <- as.numeric(niskdat_df$Temperature)
niskdat_df$NO3_NO2_USF <- as.numeric(niskdat_df$NO3_NO2_USF)
niskdat_df$SiO4_USF <- as.numeric(niskdat_df$SiO4_USF)
niskdat_df$PO4_USF <- as.numeric(niskdat_df$PO4_USF)

niskdat_df$Sigma_t <- as.numeric(niskdat_df$Sigma_t)
niskdat_df$Salinity_bottles <- as.numeric(niskdat_df$Salinity_bottles)
niskdat_df$Salinity_CTD <- as.numeric(niskdat_df$Salinity_CTD)

In [46]:
niskdat_df$O2_umol_kg <- as.numeric(niskdat_df$O2_umol_kg)
niskdat_df$POC_ug_kg <- as.numeric(niskdat_df$POC_ug_kg)
niskdat_df$PON_ug_kg <- as.numeric(niskdat_df$PON_ug_kg)
niskdat_df$PN_ug_L <- as.numeric(niskdat_df$PN_ug_L)

niskdat_df$C_N_particulate <- as.numeric(niskdat_df$C_N_particulate)
niskdat_df$PIP <- as.numeric(niskdat_df$PIP)
niskdat_df$TPP <- as.numeric(niskdat_df$TPP)
niskdat_df$DOC <- as.numeric(niskdat_df$DOC)
niskdat_df$PrimaryProductivity <- as.numeric(niskdat_df$PrimaryProductivity)
niskdat_df$Chlorophyll <- as.numeric(niskdat_df$Chlorophyll)
niskdat_df$Phaeopigments <- as.numeric(niskdat_df$Phaeopigments)


In [57]:
# Function to read specific verfication type from dataset
prepdataframe <- function(variable='', datasource='niskin'){
    if(datasource=='niskin'){
    DF <- niskdat_df
    DF$depth <- DF$Depth_real
    # print(names(DF))
        
    VarDF <- DF[complete.cases(DF[variable]),] %>%
      select(date, variable, depth) %>%
      gather(key='key',value = "value", -date, -depth)
    head(VarDF)
    }
    return(VarDF)
}

In [58]:
# integration functions (if necessary)

# adapt integration function from PINCKNEY
integrateJAYfunc <- function(x, y){
    N1 = length(y)
    area = 0
    
    for(j in 2:N1){
        area = area + (x[j] - x[j-1]) * (y[j] + y[j-1]) / 2
    }
    return(area)
}

intgrtJAY <- function(DFtest, reg, var){
    DF_int <- DFtest %>%
        group_by(date) %>%
        do(data.frame("value_int" = with(.,integrateJAYfunc(x=depth, y=value)), "reg" = reg , "var" = var)) %>%
        ungroup()
        
    return(DF_int)
}

In [59]:
# Interpolate
interpolateDF <- function(DF,func='oce-rr'){
    zz <- seq(0, 200, 1)
    
    if(func=='oce-rr'){
        IntDF <- DF %>%
            group_by(date) %>%
            do(data.frame(value_int = with(.,oceApprox(depth, value, zz, "rr")), depth = zz)) 
        }
    
    return(IntDF)
}

# take MEAN of concentrations at certain depth
meanTOdepth <- function(DF_int,type='fixed',depthFROM=0,depthTO=100){
    DF_means <- DF_int %>%
        group_by(date) %>% # this groups it by the run number 
        filter(depthFROM < depth & depth < depthTO) %>% # only keep values above the depth passed to function
        summarize('mean'=mean(value_int, na.rm=T))

    return(DF_means)}

In [68]:
Temperature <- prepdataframe('Temperature')
head(Temperature)
##############################
Temperature_int <- interpolateDF(Temperature)
##############################
Temperature_mean <- meanTOdepth(Temperature_int, type='fixed')
#head(Temperature_mean)
##############################
Temperature_atDEPTH <- meanTOdepth(Temperature_int, type='fixed', depthFROM = 100, depthTO = 150)
#head(Temperature_atDEPTH)

date,depth,key,value
1995-11-08,1.5,Temperature,27.49
1995-11-08,6.5,Temperature,27.50
1995-11-08,15.0,Temperature,27.49
1995-11-08,25.0,Temperature,26.30
1995-11-08,35.0,Temperature,25.32
1995-11-08,54.5,Temperature,24.75


# CTD DATA

In [7]:
# open a NetCDF file
ctdat <- nc_open("Data/ctd.netcdf")

#print(niskdat)

In [8]:
ctdat_df <- data.frame("Num" = 1:ctdat$dim$unlimited$len)
print('start')
for(i in 1:ctdat$nvars){
    #print(i)
    ctdat_df[paste(attributes(ctdat$var)$names[i])] <- ncvar_get(ctdat, attributes(ctdat$var)$names[i])
}
print('done reading')
head(ctdat_df)

[1] "start"
[1] "done reading"


Num,cruise_no,Cruise_ID1,Cruise_ID2,Year,Month,Day,Date,Latitude,Longitude,...,temp,sal,potemp,sigma_t,sigma_0,O2_ml_L,beam_cp,beam_att,fluor_CTD,fluor_chla
1,1,93HG_001,CAR-001,1995,11,8,1995-11-08,10.5,-64.667,...,,36.612,,,,,,,0.173,
2,1,93HG_001,CAR-001,1995,11,8,1995-11-08,10.5,-64.667,...,27.522,36.612,27.522,23.764,23.764,3.964,,,0.176,
3,1,93HG_001,CAR-001,1995,11,8,1995-11-08,10.5,-64.667,...,27.489,36.612,27.488,23.775,23.775,3.978,,,0.166,
4,1,93HG_001,CAR-001,1995,11,8,1995-11-08,10.5,-64.667,...,27.465,36.614,27.464,23.782,23.782,3.984,,,0.164,
5,1,93HG_001,CAR-001,1995,11,8,1995-11-08,10.5,-64.667,...,27.458,36.613,27.456,23.786,23.787,3.990,,,0.170,
6,1,93HG_001,CAR-001,1995,11,8,1995-11-08,10.5,-64.667,...,27.454,36.613,27.452,23.787,23.787,3.992,,,0.175,


In [9]:
cat(names(ctdat_df))

Num cruise_no Cruise_ID1 Cruise_ID2 Year Month Day Date Latitude Longitude press depth temp sal potemp sigma_t sigma_0 O2_ml_L beam_cp beam_att fluor_CTD fluor_chla

In [10]:
ctdat_df$date <- as.Date(ctdat_df$Date)

ctdat_df$press <- as.numeric(ctdat_df$press)
ctdat_df$depth <- as.numeric(ctdat_df$depth)
ctdat_df$temp <- as.numeric(ctdat_df$temp)
ctdat_df$sal <- as.numeric(ctdat_df$sal)
ctdat_df$potemp <- as.numeric(ctdat_df$potemp) 
ctdat_df$sigma_t <- as.numeric(ctdat_df$sigma_t)
ctdat_df$sigma_0 <- as.numeric(ctdat_df$sigma_0)
ctdat_df$O2_ml_L <- as.numeric(ctdat_df$O2_ml_L)
ctdat_df$beam_cp <- as.numeric(ctdat_df$beam_cp)
ctdat_df$beam_att <- as.numeric(ctdat_df$beam_att)
ctdat_df$fluor_CTD <- as.numeric(ctdat_df$fluor_CTD)
ctdat_df$fluor_chla <- as.numeric(ctdat_df$fluor_chla)

In [11]:
ctdat_df[ctdat_df>10000000000]=NA

In [61]:
CTDx258 <- ctdat_df %>%
  group_by(date) %>% # this groups it by the run number 
  #filter(press > 8) %>% # only take values below 8 meters depth to minimze influence of surface anomalies
  mutate(belowmld = sigma_t >= 25.8) %>% # create new column that gives "True" for values with sigma t higher than 25.8
  filter(belowmld == T) %>% # only take "True" values 
  slice(1) %>% # takes the first occurrence
  ungroup() #removes grouping

# PAR DATA

In [72]:
# read SeaWIFs 8Day extracted file
SeaWIFs <- read.csv('SeaWIFs_PAR_8day_extracted.csv')

In [73]:
date = as.character(paste(SeaWIFs$year,'-',SeaWIFs$month,'-15',sep=''))
SeaWIFs$Date <- as.Date(date, format='%Y-%m-%d')
str(SeaWIFs)

'data.frame':	11540 obs. of  8 variables:
 $ year : int  1997 1997 1997 1997 1997 1997 1997 1997 1997 1997 ...
 $ month: int  9 9 9 9 9 9 9 9 9 9 ...
 $ lon  : num  -64.8 -64.8 -64.7 -64.6 -64.8 ...
 $ lat  : num  10.7 10.7 10.7 10.7 10.6 ...
 $ value: num  31.1 27.3 32.1 38.9 25.7 ...
 $ unit : Factor w/ 1 level "einstein m^-2 day^-1": 1 1 1 1 1 1 1 1 1 1 ...
 $ var  : Factor w/ 1 level "par": 1 1 1 1 1 1 1 1 1 1 ...
 $ Date : Date, format: "1997-09-15" "1997-09-15" ...


# EUPHOTIC ZONE

In [78]:
# read MLD depth file
EuZ <- read.csv("par_Light_Level_edit.csv")

# read Date column
EuZ$Date <- as.Date(EuZ$Fecha, format="%m/%d/%y")

# get just month and year for merging later
EuZ$DateMonth <- format(EuZ$Date, format="%Y-%m")

In [79]:
str(EuZ)

'data.frame':	132 obs. of  7 variables:
 $ Campaign : Factor w/ 131 levels "","C118","C119",..: 72 73 74 75 76 77 78 79 80 81 ...
 $ Fecha    : Factor w/ 131 levels "","1/1/98","1/1/99",..: 23 32 2 42 54 67 89 101 111 121 ...
 $ X0.10.   : num  59 59 46.5 NA 25.5 74 71 60 71.5 74 ...
 $ X1.      : num  48.5 34 33 NA 14.5 45 45 41.5 48 57.5 ...
 $ X10.     : num  23 13.5 14 21 6.5 21 23.5 17.5 29 27 ...
 $ Date     : Date, format: "1997-11-01" "1997-12-01" ...
 $ DateMonth: chr  "1997-11" "1997-12" "1998-01" "1998-02" ...
